<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [66]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os, csv
import warnings
warnings.filterwarnings('ignore')

In [68]:
#DATAFILE_NAME = "reduced_dim_space_ddG.csv"
DATASET = "LARGE" # "LARGE" or "REDUCED"
PCA_DIMENSION = 4
if DATASET == "REDUCED":
  DATAFILE_NAME = "reduced_dim_space_ddG.csv"
  X_COLS = ['x', 'y', 'z']
else:
  DATAFILE_NAME = "merged_large_catalyst.csv"
  with open(resolve_path_gdrive("models/high_corr_cols.txt")) as f:
    reader = csv.reader(f)
    X_COLS = list(reader)[0][:-1]

df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)

In [69]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=PCA_DIMENSION)
    # fitting the model
    model.fit(X, y)
    return model

from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
print(pls_large_saved_model)

def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

## Add a predicted ddG column to the df
df['pred_ddG'] = df.apply(lambda x: pls_predict_ee(pls_large_saved_model, np.array(x[X_COLS]))[0][0], axis=1)

PLSRegression(n_components=4)


In [70]:
# start with a few randomly selected catalyst
START_SAMPLE_SIZE = 3
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
random_catalyst_df

,1,2,3,4,5,6,7,8,9,10,...,3967,3968,3969,3970,3971,3972,3973,3974,ddG,pred_ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
17_3_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194,0.003554,-0.008404
8_7_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.041476,0.495103
3_2_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.033176,0.316808


In [87]:
original_ee = df['ddG']
# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=5, cd_iterations=2, step_size=0.001):
    if pls_model == None:
        print("Creating a new model. # data items: " + str(len(catalysts)))
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where
            ## ee improves according to the model

            ## when using ddG from dataset
            #original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            original_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Found a direction for higher ddG')
    return optimized_catalysts


In [72]:
new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

In [73]:
random_catalyst_df[X_COLS]

,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
17_3_4,0.666667,0.666667,0.666667,0.666667,0.333333,0.666667,0.666667,0.666667,0.666667,0.666667,...,0.1,-0.05,-0.146,0.0,0.0,-0.05,-0.05,-0.046,0.0,0.0
8_7_1,0.000000,0.000000,0.000000,0.000000,0.068182,0.056818,0.000000,0.000000,0.005682,0.000000,...,0.0,0.00,0.000,0.0,0.0,0.00,0.00,0.000,0.0,0.0
3_2_1,0.000000,0.053333,0.120000,0.173333,0.120000,0.160000,0.040000,0.146667,0.186667,0.200000,...,0.0,0.00,0.000,0.0,0.0,0.00,0.00,0.000,0.0,0.0


In [74]:
new_optimized_catalysts

array([[ 0.66666667,  0.66666667,  0.66666667,  0.66666667,  0.35333333,
         0.66666667,  0.66666667,  0.66666667,  0.66666667,  0.68666667,
         0.68666667,  0.68666667,  0.68666667,  0.68666667,  0.66666667,
         0.66666667,  0.66666667,  0.66666667,  0.66666667,  0.66666667,
         0.68666667,  1.02      ,  1.02      ,  1.02      ,  0.68666667,
         0.68666667,  0.68666667,  0.68666667,  0.33333333,  0.68666667,
         1.02      ,  1.02      ,  0.68666667,  1.02      ,  1.02      ,
         0.68666667,  1.02      ,  1.02      ,  1.02      ,  0.68666667,
         0.35333333,  0.35333333,  0.35333333,  0.35333333,  0.68666667,
         0.02      ,  0.02      , -0.08      , -0.08      , -0.08      ,
        -0.08      , -0.08      ,  0.12      , -0.03      , -0.126     ,
         0.02      ,  0.02      , -0.03      , -0.03      , -0.026     ,
         0.02      ,  0.02      ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.08818182,
         0.05681

In [112]:
# Perform k-nearest neighbors analysis
def recomputeKNN(df, removePointsDF = None):
  if len(removePointsDF) > 0:
    df.drop(removePointsDF.index, axis=0, inplace=True)
  num_neighbors = 1  # Number of neighbors to consider
  knn = NearestNeighbors(n_neighbors=num_neighbors)
  knn.fit(df)  # Using all the catalysts
  return knn

knn = recomputeKNN(df[X_COLS], random_catalyst_df)

In [113]:
## Recursive Loop
## The method will get unique neighbors that are not already in the list.
## If the 1st neighbor is already in the list then we try the 2nd, 3rd and so on
## till a unique neighbor is found
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[[indices[0][neighbor_number-1]]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        if len(neighbor_catalyst) > 0:
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df[X_COLS]

,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
1_4_4,0.000000,0.039604,0.049505,0.039604,0.099010,0.089109,0.000000,0.009901,0.059406,0.267327,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
11_10_4,0.000000,0.000000,0.076923,0.000000,0.153846,0.076923,0.076923,0.153846,0.230769,0.076923,...,-0.011538,-0.034615,-0.034154,0.00,0.00,-0.023077,-0.034615,-0.023077,0.00,0.00
15_6_2,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,...,-0.013636,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
18_7_1,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000,0.062500,...,0.023750,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
19_11_1,0.100000,0.200000,0.200000,0.600000,0.500000,0.400000,0.200000,0.100000,0.000000,0.200000,...,0.000000,0.000000,-0.045000,0.00,0.00,0.015000,0.000000,0.060000,0.00,0.00
15_2_1,0.000000,0.040541,0.081081,0.135135,0.256757,0.121622,0.000000,0.081081,0.162162,0.175676,...,-0.004054,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
2_11_1,0.000000,0.125000,0.375000,0.500000,0.500000,0.250000,0.000000,0.000000,0.000000,0.375000,...,0.000000,0.000000,-0.018750,0.00,0.00,0.037500,0.000000,0.075000,0.00,0.00
20_7_1,0.000000,0.000000,0.000000,0.000000,0.129032,0.000000,0.000000,0.000000,0.000000,0.096774,...,-0.019355,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
15_3_1,0.545455,0.545455,0.545455,0.545455,0.545455,0.545455,0.545455,0.636364,0.636364,0.545455,...,0.040909,-0.068182,-0.093273,0.00,0.00,-0.068182,-0.068182,-0.026182,0.00,0.00


In [114]:
# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
#pls_predict_ee(pls_large_saved_model, properties1)
# random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [115]:
# We will now do all this in a single loop
max_ddG = df['ddG'].max()
print("The max value (target) of the ddG is: " + str(max_ddG))

## Get random rows
#random_catalyst_df = df.loc[not_highest_index].sample(START_SAMPLE_SIZE)
## Get lowest ddG to start
random_catalyst_df = df.sort_values('ddG').head(START_SAMPLE_SIZE)
## Static list to start
random_catalyst_df = df[df['ddG'] > .5].sort_values('ddG').head(START_SAMPLE_SIZE)
while(len(random_catalyst_df) < len(df)):
    print("The current maximum ddg is at: " + str(random_catalyst_df['pred_ddG'].max()))
    new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    # recompute knn so that we dont find the same neighbors in the next loop around
    knn = recomputeKNN(df[X_COLS], random_catalyst_df)
    if random_catalyst_df['ddG'].max() >= max_ddG:
        print("Found the highest ddG {}. Exiting.".format(str(random_catalyst_df['ddG'].max())))
        break

The max value (target) of the ddG is: 1.640249758
The current maximum ddg is at: 0.5745437642718099
The current maximum ddg is at: 0.5745437642718099
The current maximum ddg is at: 1.1848483118521564
The current maximum ddg is at: 1.1848483118521564
The current maximum ddg is at: 1.3389729931908096
Found the highest ddG 1.640249758. Exiting.
